In [ ]:
# ---------------------------------------------
# Step 1: 
    # create database in SQL server
    # create table in SQL server
# ---------------------------------------------

# create database movie_collection;

# use movie_collection
# go

# create table actor(
# 	[id] int not null,
# 	[name] varchar(50),
# 	[birthname] varchar(max),
# 	[birthdate] date,
# 	[birthplace] varchar(max),
# )

#------------------------------------------------
###you can import a JSON file to SQL server by SQL script as below:


# DECLARE @json NVARCHAR(MAX);
# SELECT @json = BulkColumn
# FROM OPENROWSET(BULK '....yourdir...\actors.json', SINGLE_CLOB) AS j;

# INSERT INTO [movie_collection].[dbo].[actor] ([id], [Name], [birthname], [birthdate], [birthplace])
# SELECT
#     ISNULL(JSON_VALUE([value], '$.id'), NULL) AS ID,
#     ISNULL(JSON_VALUE([value], '$.name'), NULL) AS [Name],
#     ISNULL(JSON_VALUE([value], '$.birthname'), NULL) AS birthname,
# 	ISNULL(cast(JSON_VALUE([value], '$.birthdate') as date), NULL) AS birthdate,
#     ISNULL(JSON_VALUE([value], '$.birthplace'), NULL) AS birthplace
# FROM OPENJSON(@json);



In [ ]:
# Step 2: install libraries python

In [ ]:
# Step 3: run below code

In [ ]:
import os
import pyodbc
import json
import pandas as pd
import warnings
from sqlalchemy import create_engine
from dateutil.parser import parse

# connection parameters
server = 'xxx'
database = 'xxxxx'
table = 'xxxxx' 
json_folder = r"xxxxxxxxxxxxxxxxx"

driver = '{ODBC Driver 17 for SQL Server}'
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes"
cnxn = pyodbc.connect(connection_string)
engine = create_engine('mssql+pyodbc://', creator=lambda: cnxn) 


# Create a connection to SQL Server (if accessing by Windows Authentication)
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes')
cursor = conn.cursor()

data_list = []

# insert all JSON files in the selected folder
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
        
            # Extract values from the JSON
            id_value = data.get('id', None)
            name = data.get('name', None)
            birthname = data.get('birthname', None)
            birthplace = data.get('birthplace', None)
            birthdate = data.get('birthdate', None)
            
            # check if birthdate is not as datetime type
            try: 
                parse(birthdate, fuzzy=False) # 👉️ (fuzzy=False) String is DateTime     
            except ValueError:
                print(name + ": " + birthdate + " is " + "False")
                break


            data_list.append([id_value, name, birthname, birthdate, birthplace])
            # print([id_value, name, birthname, birthdate, birthplace])
data_list = pd.DataFrame(data_list)
header = ['id', 'name', 'birthname', 'birthdate', 'birthplace']
data_list.columns = header
data_list['id'] = data_list['id'].apply(lambda x: int(x))

print(data_list)

data_list.to_sql(table, engine, if_exists='append', index=False)
conn.commit()
print('Finish importing')

# Close the cursor and connection
cursor.close()
conn.close()